# Python to SQL, and back again
In this codealong we will show you how to create a relational database from your pandas DataFrames.
> **To run this notebook you will need to work locally and not on colab.**

---
## 1.&nbsp; Import libraries 💾
If you haven't already installed sqlalchemy, you will need to. Uncomment the code below, install, and then recomment the code - you only need to install it once.

In [2]:
# install if needed
!pip install sqlalchemy
!pip install pymysql
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd

---
## 2.&nbsp; Relational Databases 📂

Creating DataFrames in python and pandas often results in tables with repeated information, as shown in the example below.
<br>

| author_name | book_title | year_published |
| --- | --- | --- |
| Arthur Conan Doyle | The Adventures of Sherlock Holmes | 1887 |
| J.R.R. Tolkien | The Hobbit | 1937 |
| J.R.R. Tolkien | The Lord of the Rings | 1954 |
| Harper Lee | To Kill a Mockingbird | 1960 |
| Harper Lee | Go Set a Watchman | 2015 |
<br>

This can be problematic for relational databases, which are designed to store data efficiently and avoid redundancy. To address this issue, we will separate the author and book information into two tables: authors and books. This approach eliminates duplicate data, ensuring data integrity and optimising storage.
<br>

| author_id | author_name |
| --- | --- |
| 1 | Arthur Conan Doyle |
| 2 | J.R.R. Tolkien |
| 3 | Harper Lee |
<br>

| book_id | book_title | year_published | author_id |
|---|---|---|---|
| 1 | The Adventures of Sherlock Holmes | 1887 | 1 |
| 2 | The Hobbit | 1937 | 2 |
| 3 | The Lord of the Rings | 1954 | 2 |
| 4 | To Kill a Mockingbird | 1960 | 3 |
| 5 | Go Set a Watchman | 2015 | 3 |

---
## 3.&nbsp; Creating the authors table with python 🐍
Let's start by creating the original DataFrame, including the repeated data.

In [4]:
names = ["Arthur Conan Doyle", "J.R.R. Tolkien", "J.R.R. Tolkien", "Harper Lee", "Harper Lee"]
titles = ["The Adventures of Sherlock Holmes", "The Hobbit", "The Lord of the Rings", "To Kill a Mockingbird", "Go Set a Watchman"]
years = [1887, 1937, 1954, 1960, 2015]

non_relational_df = pd.DataFrame({"author_name": names,
                                  "book_title": titles,
                                  "year_published": years})

non_relational_df

,author_name,book_title,year_published
0,Arthur Conan Doyle,The Adventures of Sherlock Holmes,1887
1,J.R.R. Tolkien,The Hobbit,1937
2,J.R.R. Tolkien,The Lord of the Rings,1954
3,Harper Lee,To Kill a Mockingbird,1960
4,Harper Lee,Go Set a Watchman,2015


Now, let's select only the authors without any duplicates.

In [5]:
authors_unique = non_relational_df["author_name"].unique()

authors_df = pd.DataFrame({"author_name": authors_unique})

authors_df

,author_name
0,Arthur Conan Doyle
1,J.R.R. Tolkien
2,Harper Lee


Fantastic! This DataFrame will be the foundation of our authors table.

---
## 4.&nbsp; Creating the matching authors table with SQL 💻

Ok, now we're ready to store this DataFrame in SQL. Before we can send the information in SQL, we need to make a table that has the same columns and data types to recieve the data. While we are creating a table for authors, we can also create the books table too.

Open MySQL Workbench, open a local connection, and open a new file. Then copy and paste the code from below.

```sql
DROP DATABASE IF EXISTS sql_workshop ;
CREATE DATABASE sql_workshop;

USE sql_workshop;

CREATE TABLE authors (
	author_id INT AUTO_INCREMENT,
    author_name VARCHAR(255),
    PRIMARY KEY (author_id)
);

CREATE TABLE books (
	book_id INT AUTO_INCREMENT,
    book_title VARCHAR(255),
    year_published VARCHAR(255),
    author_id INT,
    PRIMARY KEY (book_id),
    FOREIGN KEY (author_id) REFERENCES authors(author_id)
);
```

If you'd like more information about MySQL data types [click here](https://www.w3schools.com/mysql/mysql_datatypes.asp).

---
## 5.&nbsp; Sending the information from this notebook to sql 📠
To establish a connection with the SQL database, we need to provide the notebook with the necessary information, which we do using the connection string below. You will need to modify only the password variable, which should match the password you set during MySQL Workbench installation.

In [6]:
schema = "sql_workshop"
host = "127.0.0.1"
user = "root"
password = "Riverplate121."
port = 3306

connection_string = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'

To send information to our sql databse we use the pandas method `.to_sql()`. The argument `if_exists="append"` says that we don't want to overwrite any existing data, but add on to what is already there.

In [7]:
authors_df.to_sql('authors',
                  if_exists='append',
                  con=connection_string,
                  index=False)

3

Now, have a look at the table `authors` in MySQL Workbench, you should see that the names of the authors have appeared.

---
## 6.&nbsp; Retrieving information from sql to this notebook 📥
It's not only possible to send information to a SQL database, but also retrieve it too. Using `.read_sql()` in combination with the `connection_string` we can access the required data.

In [9]:
authors_from_sql = pd.read_sql("authors", con=connection_string)
authors_from_sql

,author_id,author_name
0,1,Arthur Conan Doyle
1,2,J.R.R. Tolkien
2,3,Harper Lee
3,4,Arthur Conan Doyle
4,5,J.R.R. Tolkien
5,6,Harper Lee


---
## 7.&nbsp; Preparing and sending the books table 📚
By extracting the authors table from our SQL database, we gain access to the unique identifier `author_id` assigned to each author. These `author_id`'s serve as pointers to their corresponding author records, allowing us to seamlessly link the `author_id`'s in the books table to their respective authors in the authors table, thereby completing the books table.

In [10]:
books_df = non_relational_df.merge(authors_from_sql,
                                   on = "author_name",
                                   how="left")

books_df

,author_name,book_title,year_published,author_id
0,Arthur Conan Doyle,The Adventures of Sherlock Holmes,1887,1
1,Arthur Conan Doyle,The Adventures of Sherlock Holmes,1887,4
2,J.R.R. Tolkien,The Hobbit,1937,2
3,J.R.R. Tolkien,The Hobbit,1937,5
4,J.R.R. Tolkien,The Lord of the Rings,1954,2
5,J.R.R. Tolkien,The Lord of the Rings,1954,5
6,Harper Lee,To Kill a Mockingbird,1960,3
7,Harper Lee,To Kill a Mockingbird,1960,6
8,Harper Lee,Go Set a Watchman,2015,3
9,Harper Lee,Go Set a Watchman,2015,6


In [11]:
books_df = books_df.drop(columns=["author_name"])

books_df

,book_title,year_published,author_id
0,The Adventures of Sherlock Holmes,1887,1
1,The Adventures of Sherlock Holmes,1887,4
2,The Hobbit,1937,2
3,The Hobbit,1937,5
4,The Lord of the Rings,1954,2
5,The Lord of the Rings,1954,5
6,To Kill a Mockingbird,1960,3
7,To Kill a Mockingbird,1960,6
8,Go Set a Watchman,2015,3
9,Go Set a Watchman,2015,6


In [52]:
books_df.to_sql('books',
                if_exists='append',
                con=connection_string,
                index=False)

5

In [53]:
books_from_sql = pd.read_sql("books", con=connection_string)
books_from_sql

,book_id,book_title,year_published,author_id
0,1,The Adventures of Sherlock Holmes,1887,1
1,2,The Hobbit,1937,2
2,3,The Lord of the Rings,1954,2
3,4,To Kill a Mockingbird,1960,3
4,5,Go Set a Watchman,2015,3


---
## 8.&nbsp; Challenge 😃
Now that you've learnt how to send and retrieve information, it's your turn to show off your skills. Create multiple tables in SQL for the data you scrapped about cities from Wikipedia. One should just be a table about the cities, the others should be facts about the cities.

| city_id | city |
| --- | --- |
| 1 | Berlin |
| 2 | Hamburg |
| 3 | Munich |

<br>

| City ID | Population | Year Data Retrieved |
|---|---|---|
| 1 | 3,850,809 | 2024 |
| 2 | 1,945,532 | 2024 |
| 3 | 1,512,491 | 2024 |

> **Pro Tip:** Visualise your relational database with pen and paper before you start coding. This can help you to identify any potential problems or inconsistencies in your design, and it can also make the coding process more efficient.

In [ ]:
#Lets bring the code from the last Web Scraping back

In [12]:

cities = ["Berlin","Hambourg","Munich","Paris","Tokyo","Barcelona","Madrid","Cologne"]
population = []
latitude = []
country = []
for i in cities:
    url_loop = f"https://en.wikipedia.org/wiki/{i}"

    response = requests.get(url_loop)

    original_soup = BeautifulSoup(response.content, 'html.parser')

    #Only using information of the lateral -> Wont return a list, as find returns a single value
    side_table = original_soup.find("table", class_="infobox ib-settlement vcard")

    #Performing the searchs on the already shortened data
    side_table.find(string = re.compile("Population")).find_next("td").get_text().replace

    population.append(int(side_table.find(string= re.compile("population", re.IGNORECASE)).find_next("td").get_text().replace(",","")))
    latitude.append(side_table.find(class_="latitude").get_text())
    country.append(side_table.find(string= re.compile("country", re.IGNORECASE)).find_next("td").get_text())

information = pd.DataFrame({"city_name":cities,"population":population,"latitude":latitude,"country":country})
information

,city_name,population,latitude,country
0,Berlin,3850809,52°31′12″N,Germany
1,Hambourg,1945532,53°33′N,Germany
2,Munich,1512491,48°08′15″N,Germany
3,Paris,2102650,48°51′24″N,France
4,Tokyo,14094034,35°41′23″N,Japan
5,Barcelona,1620343,41°22′58″N,Spain
6,Madrid,3223334,40°25′01″N,Spain
7,Cologne,1073096,50°56′11″N,Germany


In [ ]:
#Creation of a DataFrame only for Cities, then export them to SQL and import them back in order to get the cityIDs

In [312]:
cities_df = information.loc[:,"city_name"]
cities_df= pd.DataFrame(cities_df)
cities_df

,city_name
0,Berlin
1,Hambourg
2,Munich
3,Paris
4,Tokyo
5,Barcelona
6,Madrid
7,Cologne


In [147]:
schema = "gans"
host = "127.0.0.1"
user = "root"
password = "Riverplate121."
port = 3306

connection_string = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'

In [126]:
cities_df.to_sql('cities',
                  if_exists='append',
                  con=connection_string,
                  index=False)

8

In [127]:
#Returnging to Python to get city ID:
cities_from_sql = pd.read_sql("cities", con=connection_string)
cities_from_sql

,city_id,city_name
0,1,Berlin
1,2,Hambourg
2,3,Munich
3,4,Paris
4,5,Tokyo
5,6,Barcelona
6,7,Madrid
7,8,Cologne


In [315]:
#Applying same process to the table with population and latitude information
information_df = information.loc[:,["population","country","city_name"]]
information_df

,population,country,city_name
0,3850809,Germany,Berlin
1,1945532,Germany,Hambourg
2,1512491,Germany,Munich
3,2102650,France,Paris
4,14094034,Japan,Tokyo
5,1620343,Spain,Barcelona
6,3223334,Spain,Madrid
7,1073096,Germany,Cologne


In [316]:
information_df = information_df.merge(cities_from_sql,
                     left_on = "city_name",
                     right_on = "city_name",
                     how = "left")

In [317]:
information_df

,population,country,city_name,city_id
0,3850809,Germany,Berlin,1
1,1945532,Germany,Hambourg,2
2,1512491,Germany,Munich,3
3,2102650,France,Paris,4
4,14094034,Japan,Tokyo,5
5,1620343,Spain,Barcelona,6
6,3223334,Spain,Madrid,7
7,1073096,Germany,Cologne,8


In [319]:
information_df = information_df.merge(countries_from_sql,
                                      on = "country",
                                      how = "left")

In [320]:
cities_df = information.loc[:,["city_name","country"]]

In [321]:
cities_df = cities_df.merge(countries_from_sql,
                on = "country",
                how = "left")

In [322]:
cities_df = cities_df.drop("country",axis = 1)

In [323]:
cities_df

,city_name,country_id
0,Berlin,30
1,Hambourg,30
2,Munich,30
3,Paris,21
4,Tokyo,23
5,Barcelona,31
6,Madrid,31
7,Cologne,30


In [324]:
cities_df.to_sql('cities',
                  if_exists='append',
                  con=connection_string,
                  index=False)

8

In [330]:
information_df = information_df.drop(["country"],axis = 1)

In [331]:
information_df

,population,city_id
0,3850809,1
1,1945532,2
2,1512491,3
3,2102650,4
4,14094034,5
5,1620343,6
6,3223334,7
7,1073096,8


In [332]:
information_df.to_sql('information',
                  if_exists='append',
                  con=connection_string,
                  index=False)

8

In [ ]:
#Extracting amount of vehicles per 1000 habitants in the EU

In [177]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [190]:
url_loop = "https://en.wikipedia.org/wiki/List_of_countries_by_vehicles_per_capita"

response = requests.get(url_loop)

original_soup_autos = BeautifulSoup(response.content, 'html.parser')

In [201]:
#Limiting data to the Y-Axis
side_table_autos = original_soup_autos.find("tbody")

In [240]:
list_countries = side_table_autos.find_all("span", class_="flagicon")

In [290]:
#Creating list of countries and amount of vehicles
list_countries_extract = []
list_countries_extract_autos = []
for i in range(len(list_countries)):
    list_countries_extract.append(list_countries[i].find_next("a").get_text())
    list_countries_extract_autos.append(list_countries[i].find_next("td").find_next("td").get_text())
for n in range(len(list_countries_extract_autos)):
    list_countries_extract_autos[n] = list_countries_extract_autos[n].replace(",","")
    list_countries_extract_autos[n] = list_countries_extract_autos[n].replace("\n","")
    list_countries_extract_autos[n] = list_countries_extract_autos[n].replace("[","")
    list_countries_extract_autos[n] = list_countries_extract_autos[n].replace("]","")
    list_countries_extract_autos[n] = int(list_countries_extract_autos[n])

In [295]:
countries_df = pd.DataFrame({"country":list_countries_extract})

In [297]:
countries_df.to_sql('countries',
                  if_exists='append',
                  con=connection_string,
                  index=False)

195

In [301]:
autos_df = pd.DataFrame({"country":list_countries_extract,"autos":list_countries_extract_autos})

In [298]:
countries_from_sql = pd.read_sql("countries", con=connection_string)
countries_from_sql

,country_id,country
0,1,Gibraltar
1,2,Guernsey
2,3,San Marino
3,4,Liechtenstein
4,5,Andorra
...,...,...
190,191,Somalia
191,192,Sudan
192,193,Republic of the Congo
193,194,Central African Republic


In [304]:
autos_df = autos_df.merge(countries_from_sql,
                          on = "country",
                          how = "left")

In [307]:
autos_df = autos_df.drop("country",axis=1)

In [310]:
autos_df.to_sql('autos',
                  if_exists='append',
                  con=connection_string,
                  index=False)

195